# function calling
Function Calling은 AI 모델이 실시간 데이터 조회나 외부 서비스 조작을 위해 필요한 '도구'를 스스로 선택하고 실행 인자를 만들어주는 기술이다.   
모델이 함수를 직접 실행하는 대신 실행에 필요한 데이터를 구조화된 JSON 형태로 짜주면, 개발자의 코드가 이를 처리해 최신 정보를 답변에 반영할 수 있다.

### 공식문서 예시
https://platform.openai.com/docs/guides/function-calling#function-tool-example

In [ ]:
import os
import requests
import json
from dotenv import load_dotenv
from pprint import pprint

from openai import OpenAI
import json

load_dotenv()

TMDB_API_KEY = os.getenv('TMDB_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

model = 'gpt-4o-mini'


In [ ]:

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius",
                },
            },
            "required": ["sign"],
        },
    },
]

def get_horoscope(sign):
    return f"{sign}: Next Tuesday you will befriend a baby otter."

# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "What is my horoscope? I am an Aquarius."}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model=model,
    tools=tools,
    input=input_list,
)

# Save function call outputs for subsequent requests
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "get_horoscope":
            # 3. Execute the function logic for get_horoscope
            horoscope = get_horoscope(json.loads(item.arguments))
            
            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })

print("Final input:")
print(input_list)

response = client.responses.create(
    model=model,
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

### 함수 정의
LLM이 호출할 함수를 정의한다.

In [ ]:
def get_now_playing_movies(region : str = 'KR'):
    """TMDB API를 호출하여 현재 상영 중인 영화 목록을 가져옵니다."""

    print(f'{region}의 영화를 가져옵니다.')

    url = "https://api.themoviedb.org/3/movie/now_playing"
    
    params = {
        'language' : 'ko-kr',
        'region' : region
    }
    
    headers = {
        'Authorization' : f'Bearer {TMDB_API_KEY}'
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()    
        return response.json()['results']
    except Exception as e:
        print(e)

pprint(get_now_playing_movies())

### 함수 스키마 정의
LLM이 함수를 이해할 수 있도록 스키마를 정의한다.

In [ ]:
tools = [
    {
        "type": "function",
        "name": "get_now_playing_movies",
        "description": "현재 극장에서 상영 중인 영화 목록을 TMDB에서 가져옵니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "region": {
                    "type": "string",
                    "description": "region, ISO-3166-1 code",
                },

            }
        }
    }
]

In [ ]:

def run_conversation(user_prompt):
    
    input_list = [{"role": "user", "content": user_prompt}]

    # 1. LLM에게 1차 요청을 한다.
    response = client.responses.create(
        model=model,
        input=input_list,
        tools=tools,
    )
    input_list += response.output
    
    # 2. LLM의 응답에서 "함수 실행"에 대한 응답이 있다면 이를 실행한다.
    for item in response.output:
        if item.type == "function_call":
            if item.name == 'get_now_playing_movies':

                result = get_now_playing_movies(**json.loads(item.arguments))
                
                # 3. 함수 실행 결과를 input에 담는다.
                input_list.append({
                    "type": "function_call_output",
                    "call_id": item.call_id,
                    "output": json.dumps({
                    "result": result
                    })
                })
    # 4. 다시 한번 LLM에게 요청한다.
    response = client.responses.create(
        model=model,
        instructions="Respond only with a movies generated by a tool.",
        tools=tools,
        input=input_list,
    )

    return response.output_text
    

# 실행 예시
print(run_conversation("요즘 미국에서 볼만한 영화가 뭐가 있어?"))

In [4]:
import requests
from pprint import pprint

def get_news(query):
    """query에 대한 뉴스 검색 결과를 가져옵니다."""

    url = "https://google.serper.dev/news"
    
    payload = {
        "q": query,
        }
    headers = {
    'X-API-KEY': 'b0ce912249e374614d40339216224abdf2da8e75',
    'Content-Type': 'application/json'
    }

    
    try:
        response = requests.request("POST", url, headers=headers, json=payload)
        response.raise_for_status()    
        return response.json()
    except Exception as e:
        print(e)

pprint(get_news('openai'))

{'credits': 1,
 'news': [{'date': '2 hours ago',
           'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRGydFMwj9ggafrje6NsNw_ENtczFaBmpgYMjZGhPHObLBsfn0DOloiTFM&usqp=CAI&s',
           'link': 'https://www.theinformation.com/articles/openai-names-former-tml-staffer-zoph-oversee-enterprise-push',
           'snippet': 'Barret Zoph, the Thinking Machines Lab cofounder who '
                      'rejoined OpenAI in a swirl of controversy last week, '
                      "will lead the company's push to...",
           'source': 'The Information',
           'title': 'OpenAI Names Former TML Staffer Zoph to Oversee '
                    'Enterprise Push'},
          {'date': '1 day ago',
           'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTNcoHiZ8PIuJtZc3qMzAKlf0GmiD2T4FNJCtB3-KIHt1OrfZvDS2MkoGo&usqp=CAI&s',
           'link': 'https://www.reuters.com/technology/openai-rolls-out-age-prediction-chatgpt-2026-01-20/',
           'source': '